In [107]:
import pden
import pden.Net
import pden.Operations
import pden.PDENet

import tensorflow as tf

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from IPython.display import clear_output

%load_ext autoreload
%autoreload 1

%aimport pden.Net
%aimport pden.Operations
%aimport pden.PDENet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
def der(y, x, y_shape: int, x_shape: int):
    ys = tf.split(y, [1] * y_shape, 1)
    def _der(i, j=[]):
        f = ys[i]
        for _j in j:
            fs = tf.gradients(f, x)[0]
            f  = tf.split(fs, [1] * x_shape, 1)[_j]        
        return f
    return _der

In [109]:
net_u = pden.Net.BasicNet(
    pden.Operations.Linear(feature_out=4, feature_in=2, random_init = True),
    pden.Operations.ActivationFunction(tf.nn.tanh),
    pden.Operations.Linear(feature_out=4, feature_in=4, random_init = True),
    pden.Operations.ActivationFunction(tf.nn.tanh),
    pden.Operations.Linear(feature_in=4, feature_out=1, random_init = True)
)

net_v = pden.Net.BasicNet(
    pden.Operations.Linear(feature_out=4, feature_in=2, random_init = True),
    pden.Operations.ActivationFunction(tf.nn.tanh),
    pden.Operations.Linear(feature_out=4, feature_in=4, random_init = True),
    pden.Operations.ActivationFunction(tf.nn.tanh),
    pden.Operations.Linear(feature_in=4, feature_out=1, random_init = True)
)

pnet_u = pden.PDENet.PDENET(dimension_in=2, net=net_u)
pnet_v = pden.PDENet.PDENET(dimension_in=2, net=net_v)

print(net_u)
print(net_v)

Net 1:
	23	Linear: [2 -> 4]
	28	Activation funciton: <function tanh at 0x12388fc20>
	6	Linear: [4 -> 4]
	16	Activation funciton: <function tanh at 0x12388fc20>
	18	Linear: [4 -> 1]
Net 16:
	16	Linear: [2 -> 4]
	22	Activation funciton: <function tanh at 0x12388fc20>
	32	Linear: [4 -> 4]
	9	Activation funciton: <function tanh at 0x12388fc20>
	30	Linear: [4 -> 1]


In [110]:
x = tf.placeholder(tf.float64, [None, 2])
_x, _y = tf.split(x, [1, 1], 1)
u = pnet_u.forward(x)
v = pnet_v.forward(x)

bound_x_0 = tf.placeholder(tf.float64, [None, 2])
bound_x_1 = tf.placeholder(tf.float64, [None, 2])
bound_y_0 = tf.placeholder(tf.float64, [None, 2])
bound_y_1 = tf.placeholder(tf.float64, [None, 2])

bound_x_0_u = pnet_u.forward(bound_x_0)
bound_x_1_u = pnet_u.forward(bound_x_1)
bound_y_0_u = pnet_u.forward(bound_y_0)
bound_y_1_u = pnet_u.forward(bound_y_1)

bound_x_0_v = pnet_v.forward(bound_x_0)
bound_x_1_v = pnet_v.forward(bound_x_1)
bound_y_0_v = pnet_v.forward(bound_y_0)
bound_y_1_v = pnet_v.forward(bound_y_1)

ders_u = pnet_u.derivatives()
ders_v = pnet_v.derivatives()

du_dx = ders_u(0, j=[0])
dv_dx = ders_v(0, j=[0])

du_dy = ders_u(0, j=[1])
dv_dy = ders_v(0, j=[1])

$$
u = sin(x) cos(y) \\
v = cos(x) sin(y)
$$

In [111]:
pnet_u = pnet_u.add_loss(tf.reduce_mean(tf.pow(du_dx + dv_dy - (2 * tf.cos(x[:, 0]) * tf.cos(x[:, 1])), 2)))
pnet_u = pnet_u.add_loss(tf.reduce_mean(tf.pow(du_dy + dv_dx + (2 * tf.sin(x[:, 0]) * tf.sin(x[:, 1])), 2)), weight=1.0)
pnet_u = pnet_u.add_loss(tf.reduce_mean(tf.pow(bound_x_0_u - 0.0, 2)), weight=3.0)
pnet_u = pnet_u.add_loss(tf.reduce_mean(tf.pow(bound_y_0_v - 0.0, 2)), weight=3.0)

pnet_v = pnet_v.add_loss(tf.reduce_mean(tf.pow(du_dx + dv_dy - (2 * tf.cos(x[:, 0]) * tf.cos(x[:, 1])), 2)))
pnet_v = pnet_v.add_loss(tf.reduce_mean(tf.pow(du_dy + dv_dx + (2 * tf.sin(x[:, 0]) * tf.sin(x[:, 1])), 2)), weight=1.0)
pnet_v = pnet_v.add_loss(tf.reduce_mean(tf.pow(bound_x_0_u - 0.0, 2)), weight=3.0)
pnet_v = pnet_v.add_loss(tf.reduce_mean(tf.pow(bound_y_0_v - 0.0, 2)), weight=3.0)

In [120]:
learning_rate = 5e-4
training_epochs = 25001
display_step = 500

opt = tf.train.AdamOptimizer(learning_rate = learning_rate)
train = opt.minimize(pnet_u.loss + pnet_v.loss)

init = tf.global_variables_initializer()

In [121]:
n = 10
X = np.linspace(0, 1, n)

_X, _Y = np.meshgrid(X, X)
_XX = np.stack((_X.flatten(), _Y.flatten())).T

In [122]:
Y_0 = np.stack((X, np.zeros((n)))).T
Y_1 = np.stack((X, np.ones((n)))).T
X_0 = np.stack((np.zeros((n)), X)).T
X_1 = np.stack((np.ones((n)), X)).T

In [123]:
sess = tf.Session()
sess.run(init)

for epoch in range(training_epochs):
    
#     P = np.random.uniform(0, 0.5, size=(1,))
    
    _, l_u, l_v, U, V = sess.run([train, pnet_u.loss, pnet_v.loss, u, v], feed_dict={
        x: _XX,
        bound_x_0: X_0,
        bound_x_1: X_1,
        bound_y_0: Y_0,
        bound_y_1: Y_1
    })
    
    LOSS = np.mean( (U.reshape(n, n) - np.sin(_X) * np.cos(_Y)) ** 2 + (V.reshape(n, n) - np.sin(_Y) * np.cos(_X)) ** 2 )
    
    if epoch % display_step == 0:
        clear_output(wait=True)
        
        print(f'Training error for net is "{LOSS}". Epoch {epoch}')
        

        
print("Optimization Finished!")

Training error for net is "0.572255639009138". Epoch 25000
Optimization Finished!
